# Домашнее задание № 7

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [27]:
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

from nltk.tokenize import WordPunctTokenizer
from string import punctuation

punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [28]:
def normalize(sentence):
    
    sentence = [word for word in sentence.split() if word not in punct]

    return sentence

In [30]:
def compute_overlap(synset, sentence):
    gloss = set(WordPunctTokenizer().tokenize(synset.definition()))
    for i in synset.examples():
        gloss.union(i)
    # gloss = gloss.difference( functionwords )
    # sentence = sentence.difference( functionwords )
    return len(gloss.intersection(sentence))

In [31]:
def lesk(word, sentence):
    bestsense = 0
    maxoverlap = 0
    word = wn.morphy(word)
    
    for i, synset in enumerate(wn.synsets(word)):
        overlap = compute_overlap(synset, sentence)
        if overlap > maxoverlap:
            maxoverlap = overlap
            bestsense = i
        
        
    return bestsense


**Проверьте насколько хорошо работает такой метод на датасете из семинара**

In [37]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

In [348]:
corpus_wsd[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

In [360]:
# переменная для всех слов без wordnet-разметки (для функции lesk)
words = list(map(lambda a: list(map(lambda a: a.split('%')[0],
                filter(lambda s: '%' in s, chain(*a)))), corpus_wsd))
words = list(itertools.chain.from_iterable(words))
words[:10]

['long',
 'be',
 'review',
 'objective',
 'benefit',
 'service',
 'program',
 'permit',
 'become',
 'giveaway']

In [361]:
transposed = lambda item: list(map(list, zip(*item)))[-1]

In [362]:
# переменная для всех предложений (для функции lesk)
sents = list(map(
    lambda s: ' '.join(s), 
    list(map(transposed, corpus_wsd)))
            )
            

In [363]:
sents[:5]

['How long has it been since you reviewed the objectives of your benefit and service program ?',
 'Have you permitted it to become a giveaway program rather than one that has the goal of improved employee morale and , consequently , increased productivity ?',
 'What effort do you make to assess results of your program ?',
 'Do you measure its relation to reduced absenteeism , turnover , accidents , and grievances , and to improved quality and output ?',
 'Have you set specific objectives for your employee publication ?']

In [364]:
# собираем предсказания от леска
lesk_senses = []

for w, s in zip(words, sents):
    for item in w:
        try:
            lesk_senses.append(lesk(item, s))
        except AttributeError:
            continue

In [365]:
lesk_senses[:20]

[0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]

In [366]:
# достаем все слова с wordnet-разметкой, чтобы проверить точность леска на них

wordnet_tagged_words = list(map(lambda a: list(filter(lambda s: '%' in s, chain(*a))), corpus_wsd))

In [367]:
wordnet_tagged_words[0]

['long%3:00:02::',
 'be%2:42:03::',
 'review%2:31:00::',
 'objective%1:09:00::',
 'benefit%1:21:00::',
 'service%1:04:07::',
 'program%1:09:01::']

In [368]:
# собираем предсказания от wordnet
wordnet_senses = []

for w in wordnet_tagged_words:
    for ww in w:
        try:
            wordnet_senses.append(wn.lemma_from_key(ww).synset())
        except ValueError:
            continue

In [370]:
print(wordnet_senses[0])
print(wn.synsets('long'))
print(lesk_senses[0])

Synset('long.a.01')
[Synset('hanker.v.01'), Synset('long.a.01'), Synset('long.a.02'), Synset('long.s.03'), Synset('retentive.a.01'), Synset('long.a.05'), Synset('long.a.06'), Synset('long.s.07'), Synset('farseeing.s.02'), Synset('long.s.09'), Synset('long.r.01'), Synset('long.r.02')]
0


In [373]:
acc = []
for i in range(len(words)):
    try:
        acc.append(wordnet_senses[i] == wn.synsets(words[i])[lesk_senses[i]])
    except IndexError:
        acc.append(False)

#### Точность получается такая

In [374]:
(np.count_nonzero(acc)/len(acc)) * 100

7.198421716383432

### Задание 2* (2 балла)

В семинаре для WSI на данных Диалога использовался только Fastext. Попробуйте заменить его на адаграм (обучите свою модель или используйте предобученную out.pkl или https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib), а также поэкспериментируйте с разными алгоритмами кластеризации и их параметрами (можно использовать только те алгоритмы, которые обучаются достаточно быстро)

Для каждого эксперимента рассчитайте ARI

In [1]:
import adagram

In [2]:
adagram = adagram.VectorModel.load('out.pkl')

In [3]:
from sklearn.cluster import *

In [6]:
data = pd.read_csv('train.csv', sep='\t')

In [7]:
data.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,замок,1,NaN,"0-5, 339-344",замок владимира мономаха в любече . многочисле...
1,2,замок,1,NaN,"11-16, 17-22, 188-193","шильонский замок замок шильйон ( ) , известный..."
2,3,замок,1,NaN,299-304,проведения архитектурно - археологических рабо...
3,4,замок,1,NaN,111-116,"топи с . , л . белокуров легенда о завещании м..."
4,5,замок,1,NaN,"134-139, 262-267",великий князь литовский гедимин после успешной...


In [8]:
grouped_df = data.groupby('word')[['word', 'context', 'gold_sense_id']]

In [10]:
from sklearn.metrics.cluster import adjusted_rand_score

In [11]:
def get_embedding(text, model, dim):
    
    vectors = []    
    for i,word in enumerate(text):
        try:
            v = model.sense_vector(word, 0)
            vectors.append(v)
        except (KeyError, ValueError):
            continue
    vectors = np.vstack(vectors)#.reshape(-1, 3)
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

### DBScan

In [18]:
ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].apply(normalize)
    X = np.zeros((len(texts), 100))
    
    for i, text in enumerate(texts):
        text = [word for word in text if word != key]
        X[i] = get_embedding(text, adagram, 100)
        
    cluster = DBSCAN(min_samples=5, eps=0.9)
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1
    
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI))

0.1763034369313108


### KMeans

In [19]:
ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].apply(normalize)
    X = np.zeros((len(texts), 100))
    
    for i, text in enumerate(texts):
        text = [word for word in text if word != key]
        X[i] = get_embedding(text, adagram, 100)
        
    cluster = KMeans(n_clusters=10)
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1
    
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI))

0.04891654657641001


### Affinity Propagation

In [20]:
ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].apply(normalize)
    X = np.zeros((len(texts), 100))
    
    for i, text in enumerate(texts):
        text = [word for word in text if word != key]
        X[i] = get_embedding(text, adagram, 100)
        
    cluster = AffinityPropagation(damping=0.9)
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1
    
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI))

0.017448317558380694
